<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import matplotlib.pyplot as plt
import torch
# from torch.utils.data import DataLoader, TensorDataset
from pprint import pprint

from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing, ResNetProprioTunedRegression, ResNetProprioTunedSensorProcessing

from demonstration.demonstration import Demonstration
import sensorprocessing.sp_helper as sp_helper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Exp-run initialization
Create the exp/run-s that describe the parameters of the training. 
Some of the code here is structured in such a way as to make the notebook automatizable with papermill.

In [ ]:
# *** Initialize the variables with default values 
# *** This cell should be tagged as parameters     
# *** If papermill is used, some of the values will be overwritten

# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
# run = "vgg19_128"
# run = "resnet50_128"
# run = "vgg19_256"
run = "resnet50_256"

# If it is set to true, the exprun will be recreated from scratch
creation_style = "exist-ok"
# If not None, set an external experiment path
external_path = None

In [ ]:
# Parameters
epochs = 10
creation_style = "exist-ok"
run = "resnet50_128"
external_path = "/lustre/fs1/home/sa641631/WORK/BerryPicker/src/BerryPicker/src/~/WORK/BerryPicker/data_external/_automate"


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [ ]:
# create the necessary exp/run objects

if external_path:
    external_path = pathlib.Path(external_path)
    assert external_path.exists()
    Config().set_experiment_path(external_path)

exp = Config().get_experiment(experiment, run)
print(exp)

### Check the model for an encoding mode

Check whether we can load back the saved torch model. 

In [ ]:
# create the structure of the model with empty weights
if exp['model'] == 'VGG19ProprioTunedRegression':
    model = VGG19ProprioTunedRegression(exp, device)
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
else:
    raise Exception(f"Unknown model {exp['model']}")

#if exp['loss'] == 'MSELoss':        
#    criterion = nn.MSELoss()
#elif exp['loss'] == 'L1Loss':
#    criterion = nn.L1Loss()

# load the model weights
modelfile = pathlib.Path(exp.data_dir(), 
                         exp["proprioception_mlp_model_file"])
assert modelfile.exists()
model.load_state_dict(torch.load(modelfile))

In [ ]:
# Run the loaded torch model on all the specified testing data

transform = sp_helper.get_transform_to_sp(exp)

with torch.no_grad():
    for val in exp["testing_data"]:
        run, demo_name, camera = val
        exp_demo = Config().get_experiment("demonstration", run)
        demo = Demonstration(exp_demo, demo_name)
        for i in range(demo.metadata["maxsteps"]):
            sensor_readings, _ = demo.get_image(i, camera=camera, transform=transform, device=device)
            z = model.encode(sensor_readings)
            break

print(f"The shape of the output is: {z.shape}")
print(f"One example z looks like this:\n{z}")


### Create and use a SensorProcessing object

In [ ]:
# Create the sensor processing object
sp = None
if exp["model"] == "VGG19ProprioTunedRegression":
    sp = VGG19ProprioTunedSensorProcessing(exp, device)
if exp["model"] == "ResNetProprioTunedRegression":
    sp = ResNetProprioTunedSensorProcessing(exp, device)
if sp is None:
    raise Exception(f"Could not instantiate sp for {exp['model']}")

In [ ]:
# run all the testing images through the sensor processing
for val in exp["testing_data"]:
    run, demo_name, camera = val
    exp_demo = Config().get_experiment("demonstration", run)
    demo = Demonstration(exp_demo, demo_name)
    for i in range(demo.metadata["maxsteps"]):
        sensor_readings, _ = demo.get_image(i, camera=camera, transform=transform, device=device)
        z = sp.process(sensor_readings)
        break
# print the last z, as an example
print(z)
print(z.shape)